In [1]:
import data
import utils
import json
import torch
import torchvision
import numpy as np
import sklearn.metrics as skm
import torchvision.models.segmentation.deeplabv3 as dlv3
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from tqdm import tqdm

## Test-related functions

In [2]:
def eval(model, test_dataloader, tgt_num_classes, tgt_ignore_label=None):
    model.eval()
    confusion_mat = np.zeros(shape=(tgt_num_classes, tgt_num_classes), dtype=np.int64)
    
    with torch.no_grad():
        for (inputs, targets) in tqdm(test_dataloader, desc='Batch progress'):
            ipts = inputs.to('cuda', non_blocking=True)# torch.autograd.Variable(inputs).cuda()
            tgts = targets.to('cuda', non_blocking=True)# torch.autograd.Variable(targets).cuda()
            
            preds = model(ipts)['out']
            preds = torch.argmax(preds.cpu(), dim=1)
            tgts = torch.squeeze(targets, dim=1)

            confusion_mat = np.add(confusion_mat, utils.compute_confusion_matrix(predictions=preds, targets=tgts, num_classes=tgt_num_classes))
        
        miou, ious = utils.measure_performance(confusion_mat=confusion_mat,
                                               num_classes=tgt_num_classes,
                                               ignore_label=tgt_ignore_label)

    return {'miou': np.round(miou, 4), 'ious': np.round(ious, 4)}

## Test configuration loading

In [3]:
import pprint
args = {}
with open('G:/My Drive/Master IVA/Master Thesis/Models/20230417_173551_synthetic_train_check_original_real_small_dataset/eval-config.json') as json_file:
    args = json.load(json_file)
pprint.pprint(args)


{'data_source': 'real',
 'ignore_label': 19,
 'image_height': 256,
 'image_width': 512,
 'model_load_folder_path': 'G:/My Drive/Master IVA/Master '
                           'Thesis/Models/20230417_173551_synthetic_train_check_original_real_small_dataset',
 'test_batch_size': 1,
 'test_subset_size': 450,
 'train_mean': [0.28690001368522644, 0.32519999146461487, 0.2838999927043915],
 'train_std': [0.18639999628067017, 0.18970000743865967, 0.1867000013589859]}


## Data loading and preprocessing

In [4]:
test_dataset = data.HybridDataset(
        root_path=f'E:/Datasets/{args["data_source"]}/train',
        input_dir='rgb',
        target_dir='semantic_segmentation_mapped',
        ipt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"])),
                                                      torchvision.transforms.Normalize(mean=args["train_mean"], std=args["train_std"])]),
        tgt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"]))]),                        
        type=args['data_source'],
        labels_mapping=None)
test_dataset = Subset(test_dataset, indices=range(2400, 2400+args['test_subset_size']))
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=args['test_batch_size'],
    shuffle=False
)

## Model under test preparation

In [5]:
# File parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = dlv3.deeplabv3_resnet50(weights=None, progress=True, num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()))
model.load_state_dict(torch.load(f'{args["model_load_folder_path"]}/deeplabv3_model.pt'))
model.to(device)
print('Loaded model to device.')

Loaded model to device.


In [6]:
test_metrics = eval(model=model, 
                    test_dataloader=test_dataloader, 
                    tgt_num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()),
                    tgt_ignore_label=None)
test_metrics['ious'] = {k:v for k,v in zip(data.SemanticLabelMapper.ID_TO_STRING['common'].values(),
                                                   test_metrics['ious'])}


Batch progress: 100%|██████████| 450/450 [01:30<00:00,  4.98it/s]


In [7]:
print(test_metrics)
test_metrics = json.dumps(test_metrics, indent=4)
with open(f'{args["model_load_folder_path"]}/eval_metrics.json', mode='w') as f:
    f.write(test_metrics)

{'miou': 0.3852, 'ious': {'road': 0.8714, 'sidewalk': 0.5312, 'building': 0.7859, 'wall': 0.1611, 'fence': 0.1839, 'trafficlight': 0.156, 'trafficsign': 0.2437, 'vegetation': 0.8432, 'terrain': 0.4395, 'pedestrian': 0.4105, 'rider': 0.0927, 'car': 0.7991, 'truck': 0.0057, 'bus': 0.104, 'motorcycle': 0.0944, 'bicycle': 0.289, 'background': 0.537}}
